# Intro Intro

The following is an example workflow using the Chartist cli. The goal of Chartis is to provide a similar service to that of [Shileds IO](https://github.com/badges/shields)... you know, ![shield](https://img.shields.io/badge/these-things-f39f37) but for simple charts. For a run through of the basic functionality see the demo notebook here <a href="https://colab.research.google.com/github.com/tall-josh/plotypus/blob/master/notebooks/Yamgal_Demo_Notebook.ipynb" target="_blank"><img src="https://camo.githubusercontent.com/52feade06f2fecbf006889a904d221e6a730c194/68747470733a2f2f636f6c61622e72657365617263682e676f6f676c652e636f6d2f6173736574732f636f6c61622d62616467652e737667" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>. I made this with tracking and displaying Machine Learning project metrics in you git repo's README.md so that the case I'll mock you here. I'd love to see what others use if for.

---

# Regular Intro

Size of bulky png's weighing you donw? Still displaying charts as pngs like a shmuck!!!??? STOP IT, use Chartist instead.

  - No more checking in whole new pngs when you add a datapoint
  - Chartist urls track state so you don't have to
  - Choose for multiple different styles
  - Format the chart right in the url
  - Other stuff I haven't thought of yet

Say you have a project with some metrics you want to visulaize in a chart. Say you want said chart to show up in your repo's README (or any other Markdown/HTML for that matter, go nuts you crazy kids!). Typlically you'd generate a chart, save it as a png/jpg/svg or some sh💩t then check it in. With Chartist you build the data right into the url to generate the chart.

Let's get started with some imports


In [ ]:
from IPython.display import Markdown as md

# Install Chartist if running in Google Colab
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab, installing Chartist from git.')
  try:
    from chartist.chart import Chartist
  except ImportError:
    !pip install git+https://github.com/tall-josh/plotypus.git
else:
   print('Not running on CoLab, presumably you already have Chartist installed.')
   from chartist.chart import Chartist


Some variables for later

In [ ]:
ENDPOINT = 'https://yamgal-server-c6l3dwv2sq-de.a.run.app' 
README = 'README.md'
DATA = 'data.yaml'
CONFIG = 'config.yaml'


Write a sample `README.md`

Note below:

  - The existing image tags have dummy src urls, we'll fill them in later
  - the `alt` text is what Chartist uses to match when updating charts. In this case `my_metric`

In [ ]:
%%writefile {README}
# My Model

It draws boxes around shit! It draws them good. You could use Markdown or HTML
style image insert tags, we'll use HTML because this allows us to resize.

# Training History

<img src="http://something:...." alt="my_metric" height="500" width="500" >

The end

Write a sample data file to simulate the metrics you wish to visualize

In [ ]:
%%writefile {DATA}
range_name:
  - 0.9
  - 0.7
  - 0.3
  - 0.1
  - 0.02
  - 0.01

Write a simple configuration. This is optional, more info here <a href="https://colab.research.google.com/github.com/tall-josh/plotypus/blob/master/notebooks/Yamgal_Demo_Notebook.ipynb" target="_blank"><img src="https://camo.githubusercontent.com/52feade06f2fecbf006889a904d221e6a730c194/68747470733a2f2f636f6c61622e72657365617263682e676f6f676c652e636f6d2f6173736574732f636f6c61622d62616467652e737667" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In [ ]:
%%writefile {CONFIG}
title: Training Logs 
style: neon

We can generate a Chartist url using `yamgal.make_chartist_string` and insert them into the markdown text using `yamgal.find_and_replace_chart`. Add a function like this to your git pre-commit hook to automate updating your charts.

In [ ]:
import ruamel.yaml as yaml

# Put a function named `load_data` in .chartist/load_data.py
# and Chartist will load
def load_data(data_path):
    with open(data_path, 'r') as f:
        chart_data = yaml.round_trip_load(f)
    return chart_data
    
chart = Chartist.from_data_file('line',
                                DATA,
                                load_data,
                                config_path=CONFIG,
                                endpoint=ENDPOINT
                               )


We can now insert this into the Markdown document. Note:

  - Chartist attampts to make the url as short as it can by removing un-necessary zeros from floats and clipping the number of decimal points.

In [ ]:
alt_text = 'my_metric'

with open(README, 'r') as f:
    text = f.read()
    
new_text = chart.insert_into_text(text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))

After some time we get the next set metrics. We can add this to
the existing chart. 

Because the previous data is already in the existing Chartist url we can read it back into a Chartist.chart object, append the new range and write the url back into the README.

Note: Chartist recognizes ranges of the same name and add an iterator `_00, _01 ... _0n` to them accordingly.

In [ ]:
%%writefile {DATA}
range_name:
  - 0.9
  - 0.6
  - 0.1
  - 0.05
  - 0.01
  - 0.01

In [ ]:
new_data = load_data(DATA)

# load existing chart from text
chart = Chartist.from_text(new_text, alt_text)
chart.add_ranges(new_data)
new_text = chart.insert_into_text(new_text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))

You can also just append points to ranges

In [ ]:
%%writefile {DATA}
range_name_00:
  - 0.01
  - 0.01
range_name_01:
  - 0.1
  - 0.2

In [ ]:
new_points = load_data(DATA)

chart = Chartist.from_text(new_text, alt_text)
chart.append_to_ranges(new_points)
new_text = chart.insert_into_text(new_text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))